In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import timeit
import datetime
import time
import pprint
import itertools
import pickle
import sklearn
import dask
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
import fastparquet
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import h5py
import collections

/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Similarity Examples

In [51]:
ex = fastparquet.ParquetFile("DataStore/2018-06/newterm_pairs_sim_0620.parq").to_pandas()

In [54]:
ex.loc[ex["naics_name"] == 'Communications Equipment'].sample(30)

,sim_ldavecs,lead_pat,follow_pat,term,sim_docvecs,firstyear,count,follow_cites_lead,bin_sim_ldavecs,bin_sim_docvecs,naics_name
index,,,,,,,,,,,
47077,0.047598,7929537,7899092,femto,0.055915,2007,815,0,0-0.1,0.05-0.1,Communications Equipment
4994099,0.738401,6307925,5859637,wizard,0.379563,1996,327,0,0.7-0.8,>0.35,Communications Equipment
4811103,0.501903,8254277,7912040,iptv,0.228518,2005,386,0,0.5-0.6,0.2-0.25,Communications Equipment
4974034,0.338365,5446730,5644715,qos,0.285906,1993,796,0,0.3-0.4,0.25-0.3,Communications Equipment
47034,0.549306,7759607,7884446,femto,0.035854,2007,815,0,0.5-0.6,0-0.05,Communications Equipment
4994100,0.703869,6307925,5892909,wizard,0.115040,1996,327,0,0.7-0.8,0.1-0.15,Communications Equipment
47184,0.081930,8073428,7940816,femto,0.065775,2007,815,0,0-0.1,0.05-0.1,Communications Equipment
4973264,0.925633,5414469,5247363,mpeg,0.254208,1991,608,0,>0.8,0.25-0.3,Communications Equipment
4973274,0.668436,5477263,5600378,mpeg2,0.224314,1994,334,0,0.6-0.7,0.2-0.25,Communications Equipment


In [64]:
ex2 = ex.loc[(ex["lead_pat"] == 6643338)].sort_values("sim_docvecs", ascending = False)
print(len(ex2))
display(ex2.head())
ex2.sort_values("sim_docvecs")[:10]


18


,sim_ldavecs,lead_pat,follow_pat,term,sim_docvecs,firstyear,count,follow_cites_lead,bin_sim_ldavecs,bin_sim_docvecs,naics_name
index,,,,,,,,,,,
4973686,0.894032,6643338,6374383,n−1,0.373981,1998,258,0,>0.8,>0.35,Communications Equipment
4973689,0.941910,6643338,6493845,n−1,0.324950,1998,258,0,>0.8,0.3-0.35,Communications Equipment
4973691,0.975327,6643338,6563877,n−1,0.292314,1998,258,0,>0.8,0.25-0.3,Communications Equipment
4973685,0.967673,6643338,6357032,n−1,0.263427,1998,258,0,>0.8,0.25-0.3,Communications Equipment
4973695,0.485477,6643338,6591339,n−1,0.257149,1998,258,0,0.4-0.5,0.25-0.3,Communications Equipment


,sim_ldavecs,lead_pat,follow_pat,term,sim_docvecs,firstyear,count,follow_cites_lead,bin_sim_ldavecs,bin_sim_docvecs,naics_name
index,,,,,,,,,,,
4973697,0.751423,6643338,6631421,n−1,-0.026058,1998,258,0,0.7-0.8,<0,Communications Equipment
4973699,0.646861,6643338,6678375,n−1,0.035414,1998,258,0,0.6-0.7,0-0.05,Communications Equipment
4973690,0.024466,6643338,6562394,n−1,0.044020,1998,258,0,0-0.1,0-0.05,Communications Equipment
4973701,0.015069,6643338,6996820,n−1,0.051150,1998,258,0,0-0.1,0.05-0.1,Communications Equipment
4973694,0.885454,6643338,6578062,n−1,0.102461,1998,258,0,>0.8,0.1-0.15,Communications Equipment
4973696,0.002027,6643338,6615163,n−1,0.128330,1998,258,0,0-0.1,0.1-0.15,Communications Equipment
4973692,0.568611,6643338,6570683,n−1,0.130242,1998,258,0,0.5-0.6,0.1-0.15,Communications Equipment
4973700,0.556497,6643338,6724758,n−1,0.156807,1998,258,0,0.5-0.6,0.15-0.2,Communications Equipment
4973687,0.921302,6643338,6384833,n−1,0.159288,1998,258,0,>0.8,0.15-0.2,Communications Equipment


In [22]:
ex2.sort_values("sim_docvecs")

,sim_ldavecs,lead_pat,follow_pat,term,sim_docvecs,firstyear,count,follow_cites_lead,bin_sim_ldavecs,bin_sim_docvecs,naics_name
index,,,,,,,,,,,
4855611,0.007985,5586435,5644352,lun,-0.243895,1995,233,0,0-0.1,<0,Medical Equipment and Supplies
4855936,0.045310,5586435,5962322,lun,-0.226434,1995,233,0,0-0.1,<0,Medical Equipment and Supplies
4855676,0.045708,5586435,5699949,lun,-0.155272,1995,233,0,0-0.1,<0,Medical Equipment and Supplies
4855792,0.024223,5586435,5782379,lun,-0.154631,1995,233,0,0-0.1,<0,Medical Equipment and Supplies
4855925,0.020513,5586435,5931778,lun,-0.148274,1995,233,0,0-0.1,<0,Medical Equipment and Supplies
4855752,0.660478,5586435,5752506,lun,-0.148154,1995,233,0,0.6-0.7,<0,Medical Equipment and Supplies
4855860,0.178519,5586435,5832376,lun,-0.147208,1995,233,0,0.1-0.2,<0,Medical Equipment and Supplies
4855832,0.036797,5586435,5810805,lun,-0.141920,1995,233,0,0-0.1,<0,Medical Equipment and Supplies
4855976,0.046839,5586435,6336932,lun,-0.135173,1995,233,0,0-0.1,<0,Medical Equipment and Supplies


In [24]:
del(ex)

### Example 1: Patent 5586435	Hydraulic closed loop control system

Abstract 5586435: A closed loop hydraulic system especially adapted as a shot control system for a die casting machine. The control system includes a voice coil driven pilot servo valve mounted to a high flow proportional valve. The control system provides a highly controllable restriction of the outflow of hydraulic fluid from a hydraulic shot cylinder which drives a shot plunger of a die casting machine. Position transducers are provided for the pilot servo valve, proportional spool valve, and shot cylinder and a control system monitors these inputs to provide closed loop control over the shot process. A hydraulic accumulator is located in each of the pressure source and return lines of the pilot servo valve and a hydraulic accumulator tank is mounted directly to the proportional valve. The configuration of the elements of the control system along with features intended to optimize the hydraulic circuits provide a die cast shot control system with exceptionally high repeatability, frequency response and controllability coupled with low maintenance characteristics.

#### High similarity: sim docvecs 0.39

Title: Integral formed oil column extender for hydraulic lash adjuster

Abstract 5642694: A hydraulic lash adjuster for an internal combustion engine, and a method of its construction is disclosed. The lash adjuster has a cylindrical follower body in which a tubular plunger is slidingly disposed. The plunger cooperates with the follower body to define a high pressure chamber therebetween which is supplied with fluid through a valved port. A low pressure chamber which operates to supply fluid to the high pressure chamber through the valved opening, is defined by an axial passage through the plunger. Fluid flows into and out of the low pressure chamber through a port in the wall of the plunger. Intermediate of the port and the valved opening is a radially inwardly extending annular shelf which is integral formed with the axial passage of the plunger using a mechanical shaving step to plow a portion of the plunger material from the wall of the axially extending passage. The annular shelf operates as a baffle to prevent loss of fluid from the low pressure chamber when the hydraulic lash adjuster is installed at an angle from vertical.

#### Medium similarity: sim docvecs 0.1
Title: Micro-spacer metering apparatus employing multi-cavity disc and pneumatic ejection head for flat panel display assembly
Abstract 5839668: A flat panel display spacer metering apparatus comprises a motor-driven rotatable disc, having a plurality of cavities distributed around a first surface of the disc. As the disc is rotated, its cavities are filled with spacers from a storage and fill device, configured as a hollow cylinder installed in a bore of a metering block. To facilitate transfer of spacer elements stored in the storage cylinder into cavities in the disc, an axially biased plunger continuously pushes the spacer elements against the first surface of the disc. As a result, whenever rotation of the disc exposes one or more of its cavities to the open end of the cylinder, spacers are forced into and fill the cavities. As the disc is further rotated and successive ones of its cavities are filled with spacers, the filled cavities are brought into alignment with a spacer element ejection head. The ejection head continuously directs a fluid, such as dry nitrogen gas against the cavity-containing surface of the disc. As a result, when a respective cavity becomes aligned with the fluid entry port, the forced gas supplied is directed into that cavity and forces all of the spacer elements within that cavity into an exhaust port, which is coupled to the electrostatic spray gun.

#### Low similarity: sim docvecs -0.24
Title: Multicolor heat-sensitive verification and highlighting system
Abstract 5644352: A document capable of providing multiple colors under the application of heat comprising a support having at least one surface bearing a first coating comprising a substantially colorless, heat activatable chromogenic composition capable of producing a first color under the application of heat, and a second coating comprising a localized coating of a substantially colorless, heat activatable chromogenic composition capable of producing a second color under the application of heat. Each chromogenic composition comprises a chromogenic compound and a color developer that are substantially colorless solids in physical contact prior to reaction, but which can chemically react to produce a visible colored image by application of heat at temperatures above room temperature. Under the application of heat from a thermal printer, by quickly striking the chromogenic composition with a fingernail or blunt object to produce frictional heat, or other heat application, multicolored images are provided.

### Example 2: 5510842	Parallel architecture for a high definition television video decoder having multiple independent frame memories
A parallel decoder for an MPEG-2 encoded video signal includes a deformatter which separates the input bit stream into multiple portions, each representing a respectively different section of the HDTV image. The separate portions are processed in parallel by respective decoders. In order to perform motion compensated processing, each of the four decoders includes a memory that holds data representing the entire image. Each decoder provides its decoded output data to all of the decoders to maintain the data in the respective memories.

#### High Similarity (0.43)

5293229 Apparatus and method for processing groups of fields in a video data compression system

Abstract: A video compression system which is based on the image data compression system developed by the Motion Picture Experts Group (MPEG) uses various group-of-fields configurations to reduce the number of binary bits used to represent an image composed of odd and even fields of video information, where each pair of odd and even fields defines a frame. According to a first method, each field in the group of fields is predicted using the closest field which has previously been predicted as an anchor field. According to a second method, intra fields (I-fields) and predictive fields (P-fields) are distributed in the sequence so that no two I-fields and/or no two P-fields are at adjacent locations in the sequence. According to a third method, the number of I-fields and P-fields in the encoded sequence is reduced by encoding one field in a given frame as a P-field or a B-field where the other field is encoded as an I-field and encoding one field in a further frame as a B-field where the other field is encoded as a P-field.

#### Med similarity 

### Average over time
- Use NAICS Sample

In [34]:
# file_pathdir = "DataStore/2018-07-P2/Reg0726/"

# rs = fastparquet.ParquetFile(file_pathdir+"reg_naics_name_sim_tr_0726.parq").to_pandas()
# rs2 = fastparquet.ParquetFile(file_pathdir+"naics_name_common_inv_0726.parq")\
# .to_pandas(["tp", "op", "common_est_inv", "common_pat_inv", "lawyer_match"])
# rs = rs.merge(rs2, how="left", on = ["tp", "op"])
# del(rs2)

# Add year match
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq").to_pandas(["patent", "gyear"])
pdf = dict(zip(pdf["patent"], pdf["gyear"]))
rs["op_gyear"] = rs["op"].map(pdf)
rs["gyear_match"] = (rs["tp_gyear"] == rs["op_gyear"])

In [32]:
rs.columns

Index(['tp', 'op', 'sim_ldavecs', 'sim_docvecs', 'tp_gyear', 'tp_naics_name',
       'tp_primclass', 'tp_inv_msa', 'op_naics_name', 'op_primclass',
       'op_inv_msa', 'inv_msa_match', 'primclass_match', 'norm_sim_ldavecs',
       'norm_sim_docvecs', 'sim_med_ldavecs_naics_msa',
       'sim_mean_ldavecs_naics_msa', 'sim_med_docvecs_naics_msa',
       'sim_mean_docvecs_naics_msa', 'sim_med_ldavecs_pc',
       'sim_mean_ldavecs_pc', 'sim_med_docvecs_pc', 'sim_mean_docvecs_pc',
       'sim_med_ldavecs_pc_msa', 'sim_mean_ldavecs_pc_msa',
       'sim_med_docvecs_pc_msa', 'sim_mean_docvecs_pc_msa', 'year_group',
       'norm_norm_sim_ldavecs', 'norm_norm_sim_docvecs',
       'norm_sim_med_ldavecs_naics_msa', 'norm_sim_mean_ldavecs_naics_msa',
       'norm_sim_med_docvecs_naics_msa', 'norm_sim_mean_docvecs_naics_msa',
       'norm_sim_med_ldavecs_pc', 'norm_sim_mean_ldavecs_pc',
       'norm_sim_med_docvecs_pc', 'norm_sim_mean_docvecs_pc',
       'norm_sim_med_ldavecs_pc_msa', 'norm_sim_mean

In [47]:
dm = "sim_docvecs"
cd = {"common_pat_inv": "Inventor Match", "primclass_match": "Primclass Match", "gyear_match": "Year Match"}

# Same NAICS
s1 = rs[[dm, "year_group"]].groupby("year_group").mean().rename(columns={"sim_docvecs": "NAICS Match"})
# Common inventor

for k in ["gyear_match", "primclass_match", "common_pat_inv",]:
    s2 = rs.loc[(rs[k] == True), [dm, "year_group"]].groupby("year_group").mean()\
    .rename(columns={"sim_docvecs":cd[k]})
    s1 = pd.concat([s1, s2], axis=1)
    
s1 = s1.reset_index()
s1 = s1.rename(columns={"year_group": "Year Group"})

In [48]:
display(s1)

,Year Group,NAICS Match,Year Match,Primclass Match,Inventor Match
0,1975-85,0.124901,0.124883,0.189846,0.310313
1,1985-95,0.121715,0.122010,0.186469,0.334446
2,1995-05,0.127318,0.127360,0.195591,0.322542
3,2005-15,0.139251,0.139984,0.199946,0.316685


In [49]:
print(np.round(s1,3).to_latex(index=False,escape=True))

\begin{tabular}{lrrrr}
\toprule
Year Group &  NAICS Match &  Year Match &  Primclass Match &  Inventor Match \\
\midrule
   1975-85 &        0.125 &       0.125 &            0.190 &           0.310 \\
   1985-95 &        0.122 &       0.122 &            0.186 &           0.334 \\
   1995-05 &        0.127 &       0.127 &            0.196 &           0.323 \\
   2005-15 &        0.139 &       0.140 &            0.200 &           0.317 \\
\bottomrule
\end{tabular}



In [65]:
rs = fastparquet.ParquetFile(file_pathdir+"reg_primclass_sim_tr_0726.parq").to_pandas()

In [66]:
rs.columns

Index(['tp', 'op', 'sim_ldavecs', 'sim_docvecs', 'tp_gyear', 'tp_naics_name',
       'tp_primclass', 'tp_inv_msa', 'op_naics_name', 'op_primclass',
       'op_inv_msa', 'inv_msa_match', 'primclass_match', 'norm_sim_ldavecs',
       'norm_sim_docvecs', 'sim_med_ldavecs_pc_msa', 'sim_mean_ldavecs_pc_msa',
       'sim_med_docvecs_pc_msa', 'sim_mean_docvecs_pc_msa', 'year_group',
       'norm_norm_sim_ldavecs', 'norm_norm_sim_docvecs',
       'norm_sim_med_ldavecs_pc_msa', 'norm_sim_mean_ldavecs_pc_msa',
       'norm_sim_med_docvecs_pc_msa', 'norm_sim_mean_docvecs_pc_msa'],
      dtype='object')

In [67]:
rs.loc[rs["inv_msa_match"] == True, ["sim_med_ldavecs_pc_msa", "sim_med_docvecs_pc_msa"]].describe()

,sim_med_ldavecs_pc_msa,sim_med_docvecs_pc_msa
count,2.702070e+05,2.702070e+05
mean,1.000000e+00,1.000000e+00
std,7.865672e-17,8.437641e-17
min,1.000000e+00,1.000000e+00
25%,1.000000e+00,1.000000e+00
50%,1.000000e+00,1.000000e+00
75%,1.000000e+00,1.000000e+00
max,1.000000e+00,1.000000e+00
